
# Chapter 2: 데이터 준비 - OSTEP 교재 전처리 실습

이 노트북은 **Chapter 2: 데이터 준비**의 내용을 단계별로 실습할 수 있도록 구성되어 있습니다.

## 📚 학습 목표
- OSTEP 교재 데이터의 특성과 RAG 활용 방안 이해
- PDF 문서에서 텍스트 추출 및 전처리 기법 습득
- 텍스트 구조화 및 계층 정보 보존 방법 학습
- 재사용 가능한 전처리 스크립트 작성

## 📋 실습 구성
- 1️⃣ 환경 설정: 필수 패키지 설치 및 라이브러리 버전 확인
- 2️⃣ 교재 데이터 설명: OSTEP 교재 소개 및 RAG 활용 방안
- 3️⃣ 문서 파싱 및 전처리: PDF 텍스트 추출 및 불필요 요소 제거
- 4️⃣ 텍스트 정제 및 구조화: 계층 구조 파싱 및 메타데이터 추출
- 5️⃣ PDF 본문 추출 실습: 전처리 함수 구현 및 테스트

> ⚠️ *실습 셀을 실행하기 전에 반드시 환경 설정 셀을 먼저 실행하세요.*



---
## 1️⃣ 환경 설정 (Environment Setup)

RAG 시스템 구축을 위한 필수 라이브러리들을 설치하고 환경을 설정합니다. PDF 처리, 텍스트 전처리, 벡터 임베딩 등을 위한 다양한 패키지가 필요합니다.


### 1. 파이썬 가상환경 생성

독립적인 개발 환경을 위해 가상환경을 생성합니다.

```bash
$ python -m venv rag_env
```


### 2. 필수 라이브러리 설치 및 주피터 노트북 커널 설정

PDF 처리, 텍스트 전처리, 벡터 임베딩을 위한 라이브러리들을 설치합니다.

```bash
$ ./rag_env/bin/pip install -U pip
$ ./rag_env/bin/pip install jupyter ipykernel numpy pandas
$ ./rag_env/bin/pip install PyPDF2 pdfplumber pymupdf  # PDF 처리
$ ./rag_env/bin/pip install faiss-cpu langchain langchain-community  # 벡터 DB 및 RAG
$ ./rag_env/bin/pip install tiktoken openai sentence-transformers  # 임베딩
$ ./rag_env/bin/python -m ipykernel install --user --name=rag-env
```

### 3. 주피터 노트북 커널 변경

설치 완료 후 주피터 노트북에서 커널을 변경해야 합니다:

1. **Jupyter Notebook 사용 시**: 상단 메뉴에서 `Kernel` → `Change kernel` → `rag-env` 선택
2. **JupyterLab 사용 시**: 우상단 커널 이름 클릭 → `rag-env` 선택  
3. **VS Code 사용 시**: 우하단 커널 선택 버튼 클릭 → `rag-env` 선택

> 💡 **확인 방법**: 셀에서 `import sys; print(sys.executable)` 실행하여 가상환경 경로가 표시되는지 확인하세요.

---
## 2️⃣ 교재 데이터 설명: RAG에 활용할 운영체제 교재 OSTEP

OSTEP(Operating Systems: Three Easy Pieces)는 운영체제의 핵심 개념을 다루는 무료 교재로, 가상화(Virtualization), 동시성(Concurrency), 영속성(Persistence)이라는 세 가지 주요 개념을 중심으로 구성되어 있습니다. 이 교재는 명확한 설명과 실용적인 예제로 유명하며, 전 세계 많은 대학에서 운영체제 수업의 교재로 활용되고 있습니다.

RAG 시스템 구축에 OSTEP를 활용하는 이유는 교재의 체계적인 구조와 풍부한 내용 때문입니다. 각 챕터는 명확한 주제를 다루며, 개념 설명, 코드 예제, 연습문제 등이 잘 정리되어 있어 지식 베이스로 활용하기에 적합합니다. PDF 형태로 제공되는 OSTEP 교재를 파싱하여 텍스트 데이터를 추출하고, 이를 RAG 시스템의 지식 소스로 변환하는 것이 이 장의 목표입니다.


In [21]:
# OSTEP PDF 파일 경로 확인 및 기본 정보 출력
import os
import PyPDF2
from pathlib import Path

# 1. OSTEP PDF 파일 경로 확인
pdf_path = "../data/documents/ostep.pdf" 

# 파일 존재 여부 검사
if os.path.exists(pdf_path):
    actual_pdf_path = pdf_path

if actual_pdf_path:
    print(f"✅ PDF 파일 발견: {actual_pdf_path}")
    
    # 2. PDF 파일의 기본 정보 출력
    file_size = os.path.getsize(actual_pdf_path)
    print(f"📁 파일 크기: {file_size:,} bytes ({file_size/1024/1024:.2f} MB)")
    
    try:
        with open(actual_pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            num_pages = len(pdf_reader.pages)
            print(f"📄 총 페이지 수: {num_pages}")
            
            # 3. 첫 번째 챕터의 제목과 페이지 범위 확인
            print("\n🔍 첫 5페이지의 텍스트 미리보기:")
            for i in range(min(5, num_pages)):
                page = pdf_reader.pages[i]
                text = page.extract_text()
                # 첫 200자만 표시
                preview = text[:200].replace('\n', ' ').strip()
                print(f"  페이지 {i+1}: {preview}...")
                
    except Exception as e:
        print(f"❌ PDF 파일 읽기 오류: {e}")
        print("PyPDF2 대신 pdfplumber를 사용해보세요.")


✅ PDF 파일 발견: ../data/documents/ostep.pdf
📁 파일 크기: 4,147,960 bytes (3.96 MB)
📄 총 페이지 수: 643

🔍 첫 5페이지의 텍스트 미리보기:
  페이지 1: OPERATING SYSTEMS THREE EASY PIECES REMZI H. A RPACI -DUSSEAU ANDREA C. A RPACI -DUSSEAU UNIVERSITY OF WISCONSIN –M ADISON...
  페이지 2: ...
  페이지 3: .. c/circlecopyrt2014 by Arpaci-Dusseau Books, Inc. All rights reserved...
  페이지 4: ...
  페이지 5: i To Vedat S. Arpaci, a lifelong inspiration c/circlecopyrt2014, A RPACI -DUSSEAUTHREE EASY PIECES...


---
## 3️⃣ 문서 파싱 및 전처리: 교재 PDF로부터 본문 텍스트를 추출하고 불필요한 요소 제거

PDF 문서는 사람이 읽기에는 편리하지만, 기계가 처리하기에는 복잡한 형식입니다. PDF 파일은 텍스트뿐만 아니라 이미지, 폰트 정보, 레이아웃 메타데이터 등을 포함하고 있어, 순수한 텍스트 콘텐츠만을 추출하는 작업이 필요합니다. Python의 PyPDF2, pdfplumber, pymupdf 등의 라이브러리를 활용하면 PDF에서 텍스트를 추출할 수 있습니다.

추출 과정에서 발생하는 주요 문제는 페이지 머리글(header), 바닥글(footer), 쪽번호, 각주 등 본문이 아닌 요소들이 함께 추출된다는 점입니다. 이러한 불필요한 요소들은 나중에 검색 품질을 저하시킬 수 있으므로 제거해야 합니다. 정규 표현식(regex)이나 패턴 매칭을 통해 반복되는 머리글/바닥글 패턴을 찾아 제거하거나, 페이지 번호와 같은 숫자만 있는 줄을 걸러내는 방식으로 전처리를 수행합니다.

또한 PDF 추출 시 줄바꿈이나 하이픈 처리에 주의해야 합니다. PDF에서는 단어가 줄 끝에서 하이픈으로 분리되는 경우가 많고, 단락 구분이 명확하지 않을 수 있습니다. 이러한 문제를 해결하기 위해 하이픈으로 끝나는 줄을 다음 줄과 연결하고, 연속된 줄바꿈을 단락 구분으로 인식하는 등의 후처리 작업이 필요합니다.


In [36]:
# PDF에서 텍스트 추출 (PyPDF2/pdfplumber 사용)
import re
from typing import List, Dict

def extract_text_from_pdf(pdf_path: str) -> List[Dict]:
    """
    PDF에서 텍스트를 추출하고 페이지별로 정리하는 함수
    """
    pages_data = []
    
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            num_pages = len(pdf_reader.pages)
            print(f"📄 총 페이지 수: {num_pages}")
            
            for page_num, page in enumerate(pdf_reader.pages, 1):
                # 텍스트 추출
                text = page.extract_text()
                
                if text:
                    # 기본 전처리
                    cleaned_text = clean_text(text)
                    
                    pages_data.append({
                        'page_number': page_num,
                        'raw_text': text,
                        'cleaned_text': cleaned_text,
                        'char_count': len(cleaned_text)
                    })
                    
                    if page_num <= 3:  # 처음 3페이지만 미리보기
                        print(f"\n📄 페이지 {page_num} (정리 전):")
                        print(f"  원본 길이: {len(text)}자")
                        print(f"  정리 후: {len(cleaned_text)}자")
                        print(f"  미리보기: {cleaned_text[:150]}...")
                        
    except Exception as e:
        print(f"❌ PDF 처리 오류: {e}")
        
    return pages_data

def clean_text(text: str) -> str:
    """
    추출된 텍스트에서 불필요한 바닥글/머리글/페이지 번호 등을 제거하는 함수
    """
    if not text:
        return ""
    
    # 1. 하이픈으로 분리된 단어 연결
    text = re.sub(r'-\s*\n', '', text)
    
    # 2. 페이지 번호 제거 (숫자만 있는 줄)
    text = re.sub(r'^\s*\d+\s*$', '', text, flags=re.MULTILINE)
    
    # 3. 반복되는 바닥글 / 머리글 제거
    footer_patterns = [
        # "c/circlecopyrt2014..." 형태
        r'c/circlecopyrt\s*2014.*?(A\s*RPACI.*?DUSSEAU.*?THREE\s*EASY\s*PIECES)',
        r'c/circlecopyrt.*?A\s*RPACI.*?DUSSEAU.*?THREE\s*EASY\s*PIECES',
        r'©?\s*2014.*?Arpaci-?Dusseau.*?THREE\s*EASY\s*PIECES',
        
        # "OPERATING SYSTEMS [VERSION 0.80] WWW .OSTEP .ORG" 형태
        r'OPERATING\s*SYSTEMS\s*\[?VERSION\s*[\d\.]+\]?\s*WWW\s*\.?OSTEP\s*\.?ORG',
        
        # "OPERATING SYSTEMS ... THREE EASY PIECES" 형태
        r'OPERATING\s*SYSTEMS.*?THREE\s*EASY\s*PIECES',
    ]
    
    for pattern in footer_patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE | re.DOTALL)
    
    # 4. 공백 정리
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\n\s*\n', '\n\n', text)
    
    # 5. 앞뒤 공백 제거
    text = text.strip()	
    
    return text	

# 실행
if 'actual_pdf_path' in locals() and actual_pdf_path:
    print("🔄 PDF 텍스트 추출 시작...")
    pages_data = extract_text_from_pdf(actual_pdf_path)
    
    if pages_data:
        print(f"\n✅ 추출 완료: {len(pages_data)}페이지")
        
        # 전체 통계
        total_chars = sum(page['char_count'] for page in pages_data)
        avg_chars = total_chars / len(pages_data)
        print(f"📊 전체 문자 수: {total_chars:,}자")
        print(f"📊 페이지당 평균: {avg_chars:.0f}자")
        
        # 데이터 저장을 위한 변수 설정
        print(f"\n💾 pages_data 변수에 {len(pages_data)}개 페이지 데이터가 저장되었습니다.")
    else:
        print("❌ 텍스트 추출에 실패했습니다.")
else:
    print("⚠️ 먼저 PDF 파일을 찾아주세요.")


🔄 PDF 텍스트 추출 시작...
📄 총 페이지 수: 643

📄 페이지 1 (정리 전):
  원본 길이: 122자
  정리 후: 86자
  미리보기: REMZI H. A RPACI -DUSSEAU ANDREA C. A RPACI -DUSSEAU UNIVERSITY OF WISCONSIN –M ADISON...

📄 페이지 3 (정리 전):
  원본 길이: 71자
  정리 후: 71자
  미리보기: .. c/circlecopyrt2014 by Arpaci-Dusseau Books, Inc. All rights reserved...

✅ 추출 완료: 609페이지
📊 전체 문자 수: 1,302,220자
📊 페이지당 평균: 2138자

💾 pages_data 변수에 609개 페이지 데이터가 저장되었습니다.


---
## 4️⃣ 텍스트 정제 및 구조화: 추출된 텍스트를 문단/섹션 단위로 정돈하고 계층 구조 정보 보존

추출된 원시 텍스트를 그대로 사용하면 문맥 정보가 손실되고 검색 효율이 떨어집니다. 따라서 텍스트를 의미 있는 단위로 구조화하는 작업이 필요합니다. OSTEP과 같은 교재는 챕터(Chapter), 섹션(Section), 서브섹션(Subsection) 등의 계층 구조를 가지고 있으며, 이러한 구조 정보를 보존하면 나중에 더 정확한 검색과 답변 생성이 가능합니다.

구조화 작업은 제목 패턴을 인식하는 것에서 시작합니다. 교재의 챕터 제목은 보통 "Chapter 1: Introduction"과 같은 형식을 가지며, 섹션 제목은 "1.1 Virtualizing the CPU"처럼 번호가 매겨져 있습니다. 이러한 패턴을 정규 표현식으로 찾아내어 계층 구조를 파악하고, 각 텍스트 조각이 어느 챕터, 어느 섹션에 속하는지 메타데이터로 기록합니다.

문단 단위로 텍스트를 분리하는 것도 중요합니다. 연속된 두 개 이상의 줄바꿈을 문단 구분자로 인식하고, 각 문단을 독립적인 텍스트 단위로 취급합니다. 이렇게 구조화된 데이터는 JSON이나 CSV 같은 구조화된 형식으로 저장하여, 각 텍스트 조각과 함께 챕터명, 섹션명, 페이지 번호 등의 메타데이터를 함께 보관합니다. 이러한 메타데이터는 나중에 답변 생성 시 출처를 명시하는 데 활용됩니다.


In [29]:
# 챕터/섹션 구조 파싱 및 메타데이터 추출
import json
import pandas as pd
from typing import List, Dict, Optional

def parse_chapter_structure(pages_data: List[Dict]) -> List[Dict]:
    """
    페이지 데이터에서 챕터와 섹션 구조를 파싱하는 함수
    """
    structured_data = []
    current_chapter = None
    current_section = None
    
    # 정규 표현식 패턴들
    chapter_pattern = r'^Chapter\s+(\d+):\s*(.+)$'
    section_pattern = r'^(\d+\.\d+)\s+(.+)$'
    
    for page_data in pages_data:
        page_num = page_data['page_number']
        text = page_data['cleaned_text']
        
        # 페이지를 문단으로 분할
        paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
        
        for para in paragraphs:
            # 챕터 제목 확인
            chapter_match = re.match(chapter_pattern, para, re.MULTILINE)
            if chapter_match:
                chapter_num = chapter_match.group(1)
                chapter_title = chapter_match.group(2).strip()
                current_chapter = {
                    'number': int(chapter_num),
                    'title': chapter_title
                }
                current_section = None
                print(f"📚 챕터 발견: Chapter {chapter_num}: {chapter_title}")
                continue
            
            # 섹션 제목 확인
            section_match = re.match(section_pattern, para, re.MULTILINE)
            if section_match:
                section_num = section_match.group(1)
                section_title = section_match.group(2).strip()
                current_section = {
                    'number': section_num,
                    'title': section_title
                }
                print(f"  📖 섹션 발견: {section_num} {section_title}")
                continue
            
            # 일반 문단인 경우
            if len(para) > 50:  # 너무 짧은 문단은 제외
                structured_data.append({
                    'page_number': page_num,
                    'chapter_number': current_chapter['number'] if current_chapter else None,
                    'chapter_title': current_chapter['title'] if current_chapter else None,
                    'section_number': current_section['number'] if current_section else None,
                    'section_title': current_section['title'] if current_section else None,
                    'content': para,
                    'char_count': len(para),
                    'word_count': len(para.split())
                })
    
    return structured_data

def save_structured_data(data: List[Dict], output_dir: str = "output"):
    """
    구조화된 데이터를 JSON과 CSV 형식으로 저장
    """
    import os
    os.makedirs(output_dir, exist_ok=True)
    
    # JSON 저장
    json_path = os.path.join(output_dir, "ostep_structured.json")
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"💾 JSON 저장 완료: {json_path}")
    
    # CSV 저장
    csv_path = os.path.join(output_dir, "ostep_structured.csv")
    df = pd.DataFrame(data)
    df.to_csv(csv_path, index=False, encoding='utf-8')
    print(f"💾 CSV 저장 완료: {csv_path}")
    
    return json_path, csv_path

# 실행
if 'pages_data' in locals() and pages_data:
    print("🔄 챕터/섹션 구조 파싱 시작...")
    structured_data = parse_chapter_structure(pages_data)
    
    if structured_data:
        print(f"\n✅ 파싱 완료: {len(structured_data)}개 문단")
        
        # 통계 정보
        chapters = set(item['chapter_number'] for item in structured_data if item['chapter_number'])
        sections = set(item['section_number'] for item in structured_data if item['section_number'])
        
        print(f"📊 발견된 챕터 수: {len(chapters)}")
        print(f"📊 발견된 섹션 수: {len(sections)}")
        
        # 챕터별 문단 수
        chapter_counts = {}
        for item in structured_data:
            if item['chapter_number']:
                ch_num = item['chapter_number']
                chapter_counts[ch_num] = chapter_counts.get(ch_num, 0) + 1
        
        print(f"\n📚 챕터별 문단 수:")
        for ch_num in sorted(chapter_counts.keys()):
            print(f"  Chapter {ch_num}: {chapter_counts[ch_num]}개 문단")
        
        # 데이터 저장
        json_path, csv_path = save_structured_data(structured_data)
        
        # 샘플 데이터 미리보기
        print(f"\n🔍 샘플 데이터 (처음 3개 문단):")
        for i, item in enumerate(structured_data[:3]):
            print(f"\n문단 {i+1}:")
            print(f"  챕터: {item['chapter_title']} (Chapter {item['chapter_number']})")
            print(f"  섹션: {item['section_title']} ({item['section_number']})")
            print(f"  페이지: {item['page_number']}")
            print(f"  내용: {item['content'][:100]}...")
        
        print(f"\n💾 structured_data 변수에 {len(structured_data)}개 문단 데이터가 저장되었습니다.")
    else:
        print("❌ 구조 파싱에 실패했습니다.")
else:
    print("⚠️ 먼저 PDF 텍스트를 추출해주세요.")


🔄 챕터/섹션 구조 파싱 시작...

✅ 파싱 완료: 605개 문단
📊 발견된 챕터 수: 0
📊 발견된 섹션 수: 0

📚 챕터별 문단 수:
💾 JSON 저장 완료: output/ostep_structured.json
💾 CSV 저장 완료: output/ostep_structured.csv

🔍 샘플 데이터 (처음 3개 문단):

문단 1:
  챕터: None (Chapter None)
  섹션: None (None)
  페이지: 1
  내용: REMZI H. A RPACI -DUSSEAU ANDREA C. A RPACI -DUSSEAU UNIVERSITY OF WISCONSIN –M ADISON...

문단 2:
  챕터: None (Chapter None)
  섹션: None (None)
  페이지: 3
  내용: .. c/circlecopyrt2014 by Arpaci-Dusseau Books, Inc. All rights reserved...

문단 3:
  챕터: None (Chapter None)
  섹션: None (None)
  페이지: 7
  내용: Preface To Everyone Welcome to this book! We hope you’ll enjoy reading it as much a s we enjoyed wri...

💾 structured_data 변수에 605개 문단 데이터가 저장되었습니다.


---
## 5️⃣ PDF 본문 추출 실습: 예시 교재 PDF를 파싱하여 텍스트를 얻고 전처리 스크립트를 작성

실제 OSTEP PDF 파일을 대상으로 텍스트 추출 과정을 실습합니다. 먼저 PDF 파싱 라이브러리를 선택하고, 간단한 스크립트를 작성하여 한 챕터의 내용을 추출해봅니다. 예를 들어 "Introduction to Operating Systems" 챕터를 선택하여, 해당 챕터의 모든 페이지에서 텍스트를 추출하고 하나의 파일로 저장합니다.

추출된 텍스트를 육안으로 검토하면서 전처리가 필요한 부분을 파악합니다. 페이지 번호가 본문에 섞여 있는지, 머리글이나 바닥글이 반복되는지, 표나 그림의 캡션이 어떻게 추출되는지 등을 확인합니다. 이를 바탕으로 정규 표현식이나 문자열 처리 함수를 사용하여 불필요한 요소를 제거하는 전처리 함수를 작성합니다.

전처리 스크립트는 모듈화하여 재사용 가능하도록 작성합니다. PDF 파일 경로를 입력받아 정제된 텍스트를 출력하는 함수, 특정 패턴을 제거하는 함수, 계층 구조를 파싱하는 함수 등을 별도로 구현하여 조합할 수 있도록 합니다. 여러 챕터에 동일한 스크립트를 적용해보고, 각 챕터별로 추출 품질이 일관되는지 확인하여 스크립트를 개선합니다.


In [ ]:
# 전처리 함수 구현 및 테스트
import os
from pathlib import Path

class OSTEPPreprocessor:
    """
    OSTEP PDF 전처리를 위한 통합 클래스
    """
    
    def __init__(self, pdf_path: str):
        self.pdf_path = pdf_path
        self.pages_data = []
        self.structured_data = []
    
    def process_pdf(self) -> Dict:
        """
        PDF 파일을 완전히 처리하는 메인 함수
        """
        print(f"🔄 OSTEP PDF 처리 시작: {self.pdf_path}")
        
        # 1. PDF 텍스트 추출
        self.pages_data = extract_text_from_pdf(self.pdf_path)
        if not self.pages_data:
            return {"success": False, "error": "PDF 텍스트 추출 실패"}
        
        # 2. 구조 파싱
        self.structured_data = parse_chapter_structure(self.pages_data)
        if not self.structured_data:
            return {"success": False, "error": "구조 파싱 실패"}
        
        # 3. 결과 저장
        output_dir = "output"
        json_path, csv_path = save_structured_data(self.structured_data, output_dir)
        
        return {
            "success": True,
            "pages_count": len(self.pages_data),
            "paragraphs_count": len(self.structured_data),
            "json_path": json_path,
            "csv_path": csv_path
        }
    
    def get_chapter_summary(self) -> Dict:
        """
        챕터별 요약 정보 반환
        """
        if not self.structured_data:
            return {}
        
        chapter_info = {}
        for item in self.structured_data:
            if item['chapter_number']:
                ch_num = item['chapter_number']
                if ch_num not in chapter_info:
                    chapter_info[ch_num] = {
                        'title': item['chapter_title'],
                        'paragraphs': 0,
                        'total_chars': 0,
                        'sections': set()
                    }
                
                chapter_info[ch_num]['paragraphs'] += 1
                chapter_info[ch_num]['total_chars'] += item['char_count']
                
                if item['section_number']:
                    chapter_info[ch_num]['sections'].add(item['section_number'])
        
        # set을 list로 변환
        for ch_info in chapter_info.values():
            ch_info['sections'] = list(ch_info['sections'])
        
        return chapter_info
    
    def search_content(self, query: str, chapter_num: int = None) -> List[Dict]:
        """
        특정 내용을 검색하는 함수
        """
        results = []
        query_lower = query.lower()
        
        for item in self.structured_data:
            if chapter_num and item['chapter_number'] != chapter_num:
                continue
                
            if query_lower in item['content'].lower():
                results.append(item)
        
        return results

def test_preprocessing():
    """
    전처리 함수들을 테스트하는 함수
    """
    print("🧪 전처리 함수 테스트 시작...")
    
    # 테스트용 PDF 경로들
    pdf_path =  "../data/documents/ostep.pdf"
    # 전처리 실행
    preprocessor = OSTEPPreprocessor(pdf_path)
    result = preprocessor.process_pdf()
    
    if not result["success"]:
        print(f"❌ 전처리 실패: {result['error']}")
        return False
    
    print(f"✅ 전처리 성공!")
    print(f"  📄 처리된 페이지: {result['pages_count']}개")
    print(f"  📝 추출된 문단: {result['paragraphs_count']}개")
    print(f"  💾 JSON 파일: {result['json_path']}")
    print(f"  💾 CSV 파일: {result['csv_path']}")
    
    # 챕터별 요약
    print(f"\n📚 챕터별 요약:")
    chapter_summary = preprocessor.get_chapter_summary()
    for ch_num in sorted(chapter_summary.keys()):
        ch_info = chapter_summary[ch_num]
        print(f"  Chapter {ch_num}: {ch_info['title']}")
        print(f"    - 문단 수: {ch_info['paragraphs']}개")
        print(f"    - 총 문자 수: {ch_info['total_chars']:,}자")
        print(f"    - 섹션 수: {len(ch_info['sections'])}개")
    
    # 검색 테스트
    print(f"\n🔍 검색 테스트:")
    test_queries = ["process", "memory", "scheduling"]
    for query in test_queries:
        results = preprocessor.search_content(query)
        print(f"  '{query}' 검색 결과: {len(results)}개 문단")
        if results:
            sample = results[0]
            print(f"    샘플: {sample['content'][:100]}...")
    
    return True

# 실행
if __name__ == "__main__":
    test_preprocessing()


🧪 전처리 함수 테스트 시작...
🔄 OSTEP PDF 처리 시작: ../data/documents/ostep.pdf
📄 총 페이지 수: 643

📄 페이지 1 (정리 전):
  원본 길이: 122자
  정리 후: 86자
  미리보기: REMZI H. A RPACI -DUSSEAU ANDREA C. A RPACI -DUSSEAU UNIVERSITY OF WISCONSIN –M ADISON...

📄 페이지 3 (정리 전):
  원본 길이: 71자
  정리 후: 71자
  미리보기: .. c/circlecopyrt2014 by Arpaci-Dusseau Books, Inc. All rights reserved...
💾 JSON 저장 완료: output/ostep_structured.json
💾 CSV 저장 완료: output/ostep_structured.csv
✅ 전처리 성공!
  📄 처리된 페이지: 609개
  📝 추출된 문단: 605개
  💾 JSON 파일: output/ostep_structured.json
  💾 CSV 파일: output/ostep_structured.csv

📚 챕터별 요약:

🔍 검색 테스트:
  'process' 검색 결과: 239개 문단
    샘플: v To Educators If you are an instructor or professor who wishes to use this bo ok, please feel free ...
  'memory' 검색 결과: 270개 문단
    샘플: v To Educators If you are an instructor or professor who wishes to use this bo ok, please feel free ...
  'scheduling' 검색 결과: 78개 문단
    샘플: xii CONTENTS 5 Interlude: Process API 35 5.1 Thefork() System Call . . . . . . . . . . . . . . . . ....
